<a href="https://www.kaggle.com/code/mesutssmn/dog-breed-cnn-nasnetlarge?scriptVersionId=166354813" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_dir = "/kaggle/input/dog-breed-identification/train"
test_dir = "/kaggle/input/dog-breed-identification/test"

In [ ]:
train_img = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
test_img = [os.path.join(test_dir, f) for f in os.listdir(test_dir)]

In [ ]:
df_train = pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")
df_train.head()
len(df_train.breed.unique())

In [ ]:
df_train.rename(columns = {"breed": "label"}, inplace = True)
df_train["id"] = df_train["id"].apply(lambda x: x+"."+"jpg")
df_train.head()


In [ ]:
test_files = os.listdir(test_dir)
df_test = pd.DataFrame({"id": test_files, "label": "dingo"})
df_test.head()

In [ ]:
src_path = "../input/dog-breed-identification/train"
sub_class = os.listdir(src_path)

fig = plt.figure(figsize=(10,5))
for e in range(len(sub_class[:8])):
    plt.subplot(2,4,e+1)
    img = plt.imread(os.path.join(src_path,sub_class[e]))
    plt.imshow(img, cmap=plt.get_cmap('gray'))
    plt.axis('off')

In [ ]:
train_size = len(df_train)
test_size = len(df_test)

train_size,test_size

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=25,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(df_train, train_dir,
    x_col = 'id',
    y_col = 'label',
    subset = 'training',
    color_mode = 'rgb',
    class_mode = 'categorical',
    target_size = (224, 224),
    batch_size = 32,
    shuffle = True,
    seed = 1881)

In [ ]:
validation_generator = train_datagen.flow_from_dataframe(df_train, train_dir,
    x_col = 'id',
    y_col = 'label',
    subset = 'validation',
    color_mode = 'rgb',
    class_mode = 'categorical',
    target_size = (224, 224),
    batch_size = 32,
    shuffle = True,
    seed = 1881)


In [ ]:
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory=test_dir,
x_col="id",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224),
color_mode="rgb")

In [ ]:
conv_model = tf.keras.applications.NASNetLarge(
                  weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3)
               )
conv_model.trainable = False
conv_model.summary()

In [ ]:
model = tf.keras.models.Sequential()

model.add(conv_model)

model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(120, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=['acc'])

model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                          min_delta = 0.001,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)]

In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    callbacks=callbacks,
    validation_data=validation_generator,
    batch_size=40)

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('MODEL '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'LOSS',
    211,
)

display_training_curves(
    history.history['acc'],
    history.history['val_acc'],
    'ACCURACY',
    212,
)

In [ ]:
score = model.evaluate(validation_generator,batch_size=32)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

In [ ]:
pred=model.predict(test_generator)

In [ ]:
df_submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
df_submission.head()

In [ ]:
df_submission.iloc[:,1:] = pred
df_submission.head()

In [ ]:
final_df = df_submission.set_index('id')
final_df.to_csv('Submission.csv')